In [1]:
from __future__ import print_function
import cobra.test
from cobra.flux_analysis import parsimonious

model = cobra.test.create_test_model("ecoli")
metabolite_ids = [m.id for m in model.metabolites]
print('Model %s' % model.description)

Model iJO1366


In [2]:
def get_redox_reactions(pair, exclude_metabolites=None):
    """Grabs the redox reactions in which this pair participates.
    
    Args:
        pair: 2 tuple of oxidized and reduced form of a redox cofactor.
            order irrelevant.
        exclude_metabolites: an iterable of metabolites.
            if any of these appear in a reaction, the reaction will be
            excluded from the returned list.
    
    Returns:
        A list of reaction IDs.
    """
    first, second = pair
    first_reactions = first.reactions
    second_reactions = second.reactions
    
    if exclude_metabolites:
        exclude_set = set(exclude_metabolites)
        first_reactions = [r for r in first_reactions
                           if not set(r.metabolites.keys()).intersection(exclude_set)]
        second_reactions = [r for r in second_reactions
                            if not set(r.metabolites.keys()).intersection(exclude_set)]
    
    rids = set([r.id for r in first_reactions])
    rids.intersection_update([r.id for r in second_reactions])
    return rids
    

In [3]:
# O2
o2 = model.metabolites.get_by_id('o2_c')
exclude = [o2]

# NAD(P)(H)
nad = model.metabolites.get_by_id('nad_c')
nadh = model.metabolites.get_by_id('nadh_c')
nadp = model.metabolites.get_by_id('nadp_c')
nadph = model.metabolites.get_by_id('nadph_c')

nad_reactions = get_redox_reactions((nad, nadh), exclude)
nadp_reactions = get_redox_reactions((nadp, nadph), exclude)
all_nad_reactions = nad_reactions.union(nadp_reactions)

# FAD(H)
fad = model.metabolites.get_by_id('fad_c')
fadh = model.metabolites.get_by_id('fadh2_c')
fad_reactions = get_redox_reactions((fad, fadh), exclude)

# Ferredoxin is annotated as iron 2/3
fd_ox = model.metabolites.get_by_id('fe3_c')
fd_red = model.metabolites.get_by_id('fe2_c')
fd_reactions = get_redox_reactions((fd_ox, fd_red), exclude)

# Quinones
menaquinone_8 = model.metabolites.get_by_id('mqn8_c')
menaquinol_8 = model.metabolites.get_by_id('mql8_c')
ubiquinone_8 = model.metabolites.get_by_id('q8_c')
ubiquinol_8 = model.metabolites.get_by_id('q8h2_c')

mena_reactions = get_redox_reactions((menaquinone_8, menaquinol_8), exclude)
ubi_reactions = get_redox_reactions((ubiquinone_8, ubiquinol_8), exclude)

# Disulfide equivalents glutathione, thioredoxin and glutaredoxin
glth_ox = model.metabolites.get_by_id('gthox_c')
glth_red = model.metabolites.get_by_id('gthrd_c')
thio_ox = model.metabolites.get_by_id('trdox_c')
thio_red = model.metabolites.get_by_id('trdrd_c')
glut_ox = model.metabolites.get_by_id('grxox_c')
glut_red = model.metabolites.get_by_id('grxrd_c')

glth_reactions = get_redox_reactions((glth_ox, glth_red), exclude)
thio_reactions = get_redox_reactions((thio_ox, thio_red), exclude)
glut_reactions = get_redox_reactions((glut_ox, glut_red), exclude)

non_nad_rxn_sets = [fad_reactions, fd_reactions, mena_reactions, ubi_reactions,
                    glth_reactions, thio_reactions, glut_reactions]
non_nad_reactions = set().union(*non_nad_rxn_sets)

n_nad = len(all_nad_reactions)
n_non_nad = len(non_nad_reactions)
total_redox_reactions = n_nad + n_non_nad
print('%d reactions using NAD(P) as redox carrier' % n_nad)
print('%d reactions using something other than NAD(P) as redox carrier' % n_non_nad)
print('%d reactions total' % total_redox_reactions)

frac = n_nad / float(total_redox_reactions)
print('Fraction of reactions using NAD(P) %.2f' % frac)

205 reactions using NAD(P) as redox carrier
87 reactions using something other than NAD(P) as redox carrier
292 reactions total
Fraction of reactions using NAD(P) 0.70


In [4]:
def solve_w_csource(model, csource_rxn_id, anaerobic=False):
    """Sets the given reaction ID as the carbon source and solves.
    
    Solves using parsimonious FBA.
    
    Args:
        model: metabolic model.
        csource_rxn_id: ID of the import reaction for the carbon source. 
    
    Returns:
        Optimization solution.
    """
    rxn = model.reactions.get_by_id(csource_rxn_id)
    rxn.lower_bound = -10.0
    
    oxygen_uptake = model.reactions.get_by_id('EX_o2_e')
    if anaerobic:
        oxygen_uptake.lower_bound = 0
        oxygen_uptake.upper_bound = 0
    else:
        oxygen_uptake.lower_bound = -1000
        oxygen_uptake.upper_bound = 1000
    
    try:
        res = parsimonious.optimize_minimal_flux(model)
    except Exception as e:
        res = None
        
    # Reset lower bound after solving
    rxn.lower_bound = 0.0
    
    return res
    
    
def print_redox_fluxes(solution):
    """Given a model solution, print the redox reaction fluxes."""
    if solution is None:
        print('Model could not be solved')
        return
    
    fluxes_dict = solution.x_dict
    nad_fluxes = [abs(fluxes_dict.get(r)) for r in all_nad_reactions]
    total_nad_flux = sum(nad_fluxes)
    print('Total NAD flux', total_nad_flux)
    
    non_nad_fluxes = [abs(fluxes_dict.get(r)) for r in non_nad_reactions]
    total_non_nad_flux = sum(non_nad_fluxes)
    print('Total Non-NAD flux', total_non_nad_flux)

    fraction_of_total = total_nad_flux / (float(total_nad_flux + total_non_nad_flux))
    print('Fraction of total %.2f' % fraction_of_total)

In [5]:
# Set external glucose to 0 so we can test different csources.
# Glucose is the default carbon source. 
glucose_uptake = model.reactions.get_by_id('EX_glc_e')
glucose_uptake.lower_bound = 0.0

csources = {
  'sixc': {'glucose': 'EX_glc_e',
           'gluconate': 'EX_glcn_e',
           'galactose': 'EX_gal_e'},
  'fivec': {'ribose': 'EX_rib__D_e',
            'rhamnose': 'EX_rmn_e',
            'xylose': 'EX_xyl__D_e'},
  'fourc': {'malate': 'EX_mal__D_e',
            'succinate': 'EX_succ_e',
            'acetoacetate': 'EX_acac_e'},
  'threec': {'glycerol': 'EX_glyc_e',
             'pyruvate': 'EX_pyr_e'},
  'twoc': {'glycine': 'EX_glcn_e',
           'acetate': 'EX_ac_e',
           'acetaladehyde': 'EX_acald_e'}}

for anaerobic in (True, False):
    oxy_tag = 'anaerobic' if anaerobic else 'aerobic'
    for carbons, csource_dict in csources.iteritems():
        print('#### %s substrates (%s metabolism)' % (carbons, oxy_tag))
        for name, rxn_id in csource_dict.iteritems():
            soln = solve_w_csource(model, rxn_id, anaerobic=anaerobic)
            print('Fluxes for %s (%s)' % (name, oxy_tag))            
            print_redox_fluxes(soln)
            print()

#### fourc substrates (anaerobic metabolism)
Fluxes for succinate (anaerobic)
Model could not be solved

Fluxes for acetoacetate (anaerobic)
Total NAD flux 5.17292899996
Total Non-NAD flux 0.30930419467
Fraction of total 0.94

Fluxes for malate (anaerobic)
Model could not be solved

#### twoc substrates (anaerobic metabolism)
Fluxes for acetaladehyde (anaerobic)
Total NAD flux 21.6727115334
Total Non-NAD flux 0.256421173019
Fraction of total 0.99

Fluxes for acetate (anaerobic)
Model could not be solved

Fluxes for glycine (anaerobic)
Total NAD flux 49.9692836968
Total Non-NAD flux 1.26918854212
Fraction of total 0.98

#### threec substrates (anaerobic metabolism)
Fluxes for pyruvate (anaerobic)
Total NAD flux 21.5385137688
Total Non-NAD flux 0.403790213804
Fraction of total 0.98

Fluxes for glycerol (anaerobic)
Total NAD flux 60.5364561628
Total Non-NAD flux 0.63178869094
Fraction of total 0.99

#### fivec substrates (anaerobic metabolism)
Fluxes for xylose (anaerobic)
Total NAD flux 